In [75]:
import numpy as np
import pandas as pd 
import matplotlib.pyplot as plt
import pandas_profiling
import requests
import json
import time
import plotly.plotly as py

In [86]:
try:
    rj2 = pd.read_csv('../../datasets/eleicao_rj/limpo.csv', index_col=0)
except FileNotFoundError:
    df = pd.read_csv('../../datasets/eleicao_rj/votacao_secao_2018_RJ.csv',
                    sep=';', encoding='Latin1')
    rj = df.query('DS_CARGO == "GOVERNADOR" and NM_UE == "RIO DE JANEIRO"')
    del df
    rj2 = rj.drop(['DT_GERACAO', 'HH_GERACAO', 'ANO_ELEICAO', 'CD_TIPO_ELEICAO',
            'NM_TIPO_ELEICAO', 'NR_TURNO', 'CD_ELEICAO', 'DS_ELEICAO',
            'DT_ELEICAO', 'TP_ABRANGENCIA', 'CD_MUNICIPIO', 'SG_UF', 'SG_UE',
            'NM_UE', 'CD_CARGO', 'DS_CARGO'],
           axis=1)
    rj2.columns = ['municipio', 'zona', 'secao', 'nr_candidato', 'candidato',
                  'votos']
    rj2.to_csv('../../datasets/eleicao_rj/limpo.csv')
    del rj    

In [87]:
rj2

,municipio,zona,secao,nr_candidato,candidato,votos
81,RIO DE JANEIRO,170,423,55,ANTONIO PEDRO INDIO DA COSTA,8
149,BELFORD ROXO,153,151,50,TARCISIO MOTTA DE CARVALHO,8
156,RIO DE JANEIRO,238,24,96,VOTO NULO,54
164,TERESÓPOLIS,38,130,96,VOTO NULO,33
178,MACAÉ,254,147,50,TARCISIO MOTTA DE CARVALHO,10
183,RIO DE JANEIRO,216,129,50,TARCISIO MOTTA DE CARVALHO,24
186,RIO DE JANEIRO,4,242,50,TARCISIO MOTTA DE CARVALHO,79
188,RIO DE JANEIRO,9,271,50,TARCISIO MOTTA DE CARVALHO,42
196,NOVA IGUAÇU,84,625,50,TARCISIO MOTTA DE CARVALHO,15
197,RIO DE JANEIRO,122,310,96,VOTO NULO,51


In [3]:
pandas_profiling.ProfileReport(rj2)

Number of variables,7
Number of observations,389830
Total Missing (%),0.0%
Total size in memory,20.8 MiB
Average record size in memory,56.0 B
Numeric,5
Categorical,2
Boolean,0
Date,0
Text (Unique),0
Rejected,0


## Limpeza dos dados das zonas eleitorais

In [71]:
end_zonas = pd.read_csv('../../datasets/eleicao_rj/lista_zonas_eleitorais.csv',
                       encoding='Latin1', index_col=0)
end_zonas = end_zonas.rename(columns={'endereco': 'rua'})
end_zonas = end_zonas.drop(['sigla_uf', 'cep', 'sigla_uf.1'], axis=1)
end_zonas['endereco'] = end_zonas.apply(lambda x: ', '.join(x), axis=1)

Vamos utilizar o serviço do google para geocodificar os endereços das zonas caso não exista o arquivo pronto.

In [72]:
erros = []
try:
    end_zonas = pd.read_csv('../../datasets/eleicao_rj/zonas_eleitorais_proc.csv', index_col=0)
except FileNotFoundError:
    dados_json = []
    url = 'https://maps.googleapis.com/maps/api/geocode/json'
    bounds = '-23.371469,-44.917957|-20.784440,-40.504246'
    with open('gkey.txt') as f:
        g_key = f.read()
    for z,v in end_zonas.iterrows():
        end = v["endereco"].replace(' ', '+')
        response = requests.get(url, {'bounds': bounds, 'key': g_key, 'address': end})
        try:
            coords = json.loads(response.text)['results'][0]['geometry']['location']
            end_zonas.loc[z, 'lat'] = coords['lat']
            end_zonas.loc[z, 'lng'] = coords['lng']
            dados_json.append(response.json())
        except:
            erros.append(z)
            print(f'Não achou zona {z}')
        time.sleep(0.05)
    end_zonas.loc[60, 'lat'] = -21.954952
    end_zonas.loc[60, 'lng'] = -42.136810 
    end_zonas.loc[196, 'lat'] = -22.151659
    end_zonas.loc[196, 'lng'] = -42.926066    
    end_zonas.index.name = 'zona'
    end_zonas.to_csv('../../datasets/eleicao_rj/zonas_eleitorais_proc.csv')

Não achou zona 60


## Limpeza dos dados dos votos
Vamos criar dois datasets diferentes, um contendo o percentual dos votos de cada zona eleitoral foi para cada candidato e o outro contendo a representatividade de votos de cada zona eleitoral em relação ao total de cada candidato.

In [68]:
votos = rj2.loc[:,['zona', 'candidato', 'votos']].set_index('zona')
votos_zona = np.sum(votos.groupby('zona'))['votos']
votos_candidato = np.sum(votos.groupby('candidato'))
votos = np.sum(votos.groupby(['zona', 'candidato']))
votos_perc_zona = pd.DataFrame(columns=list(votos.index.levels[1]))
votos_perc_cand = pd.DataFrame(columns=list(votos.index.levels[1]))
for z in votos_zona.index.values:
    for i, v in (votos.loc[z]/votos_zona[z]).iterrows():
        votos_perc_zona.loc[z, i] = round(v[0]*100, 2)
    for i, v in (votos.loc[z]/votos_candidato).iterrows():
        votos_perc_cand.loc[z, i] = round(v[0]*100, 2)
votos_perc_cand.index.name = 'zona'
votos_perc_zona.index.name = 'zona'

In [71]:
votos_perc_cand.to_csv('../../datasets/eleicao_rj/votos_perc_cand.csv')
votos_perc_zona.to_csv('../../datasets/eleicao_rj/votos_perc_zona.csv')

## Enriquecendo os dados

A partir dos dados do IDH Municipal, agregaremos esses dados na nossa análise de forma para enriquecermos a nossa análise.

In [77]:
idhm = pd.read_excel('../../datasets/IDHM/IDHM_Municipios_RJ.xlsx',
                     index_col=0)
idhm.head(1)

,Espacialidades,IDHM 2010
Código,,
76,Brasil,0.727


In [78]:
idhm

,Espacialidades,IDHM 2010
Código,,
76,Brasil,0.727
3300100,Angra dos Reis,0.724
3300159,Aperibé,0.692
3300209,Araruama,0.718
3300225,Areal,0.684
3300233,Armação dos Búzios,0.728
3300258,Arraial do Cabo,0.733
3300308,Barra do Piraí,0.733
3300407,Barra Mansa,0.729
